# SCIBERT + SHAP

In [ ]:
import shap
import spacy

In [ ]:
# load the model
best_model = f"models/ft-scibert-zora/model-best"
spacy_model = spacy.load(best_model)

# clean energy belongs to class '7'
spacy_probs = [doc.cats for doc in spacy_model.pipe(["clean energy"])]
spacy_probs

In [ ]:
tokenizer_spacy = spacy.tokenizer.Tokenizer(spacy_model.vocab)

# Run the spacy pipeline on some random text just to retrieve the classes
doc = spacy_model()
classes = list(doc.cats.keys())

# Define a function to predict
def predict(texts):
    # convert texts to bare strings
    texts = [str(text) for text in texts]
    results = []
    for doc in spacy_model.pipe(texts):
        # results.append([{'label': cat, 'score': doc.cats[cat]} for cat in doc.cats])
        results.append([doc.cats[cat] for cat in classes])
    return results

# Create a function to create a transformers-like tokenizer to match shap's expectations
def tok_adapter(text, return_offsets_mapping=False):
    doc = tokenizer_spacy(text)
    out = {"input_ids": [tok.norm for tok in doc]}
    if return_offsets_mapping:
        out["offset_mapping"] = [(tok.idx, tok.idx + len(tok)) for tok in doc]
    return out


In [ ]:
explainer = shap.Explainer(predict, masker=shap.maskers.Text(tok_adapter), algorithm="permutation", output_names=classes, max_evals=1500)


In [ ]:
samples = [
    "Recent Advances and Emerging Trends in Photo-Electrochemical Solar Energy Conversion. Photo‐electrochemical (PEC) solar energy conversion offers the promise of low‐cost renewable fuel generation from abundant sunlight and water. In this Review, recent developments in photo‐electrochemical water splitting are discussed with respect to this promise. State‐of‐the‐art photo‐electrochemical device performance is put in context with the current understanding of the necessary requirements for cost‐effective solar hydrogen generation (in terms of solar‐to‐hydrogen conversion efficiency and system durability, in particular). Several important studies of photo‐electrochemical hydrogen generation at p‐type photocathodes are highlighted, mostly with protection layers (for enhanced durability), but also a few recent examples where protective layers are not needed. Recent work with the widely studied n‐type BiVO4 photoanode is detailed, which highlights the needs and necessities for the next big photoanode material yet to be discovered. The emerging new research direction of photo‐electrocatalytic upgrading of biomass substrates toward value‐added chemicals is then discussed, before closing with a commentary on how research on PEC materials remains a worthwhile endeavor.",
    "Ruthenium Water Oxidation Catalysts based on Pentapyridyl Ligands. Ruthenium complexes containing the pentapyridyl ligand 6,6′′-(methoxy(pyridin-2-yl)methylene)di-2,2′-bipyridine (L-OMe) of general formula trans-[RuII(X)(L-OMe-κ-N5)]n+ (X=Cl, n=1, trans-1+; X=H2O, n=2, trans-22+) have been isolated and characterized in solution (by NMR and UV/Vis spectroscopy) and in the solid state by XRD. Both complexes undergo a series of substitution reactions at oxidation state RuII and RuIII when dissolved in aqueous triflic acid–trifluoroethanol solutions as monitored by UV/Vis spectroscopy, and the corresponding rate constants were determined. In particular, aqueous solutions of the RuIII-Cl complex trans-[RuIII(Cl)(L-OMe-κ-N5)]2+ (trans-12+) generates a family of Ru aquo complexes, namely trans-[RuIII(H2O)(L-OMe-κ-N5)]3+ (trans-23+), [RuIII(H2O)2(L-OMe-κ-N4)]3+ (trans-33+), and [RuIII(Cl)(H2O)(L-OMe-κ-N4)]2+ (trans-42+). Although complex trans-42+ is a powerful water oxidation catalyst, complex trans-23+ has only a moderate activity and trans-33+ shows no activity. A parallel study with related complexes containing the methyl-substituted ligand 6,6′′-(1-pyridin-2-yl)ethane-1,1-diyl)di-2,2′-bipyridine (L-Me) was carried out. The behavior of all of these catalysts has been rationalized based on substitution kinetics, oxygen evolution kinetics, electrochemical properties, and density functional theory calculations. The best catalyst, trans-42+, reaches turnover frequencies of 0.71 s−1 using CeIV as a sacrificial oxidant, with oxidative efficiencies above 95 %."
]
# Process the text using SpaCy
doc = spacy_model(samples[0])
# Get the shap values
shap_values = explainer([samples[0]])
shap.plots.text(shap_values)

In [ ]:
html = shap.plots.text(shap_values, display=False)

# save html to "shap.html"
with open("shap.html", "w") as f:
    f.write(html)

In [ ]:
# what is the output, hyperparams, where does tokenization happen (cased vs uncased), know what the tokenization, periods punctioations
# overview of the dataflow of spacy and shap
# (name) what type of the graph is used for shap
# understand the underlying data structure (shap), typical lexical ways to indicate a certain sdg
# semantic similarity search on best features (tokens) per class
# least effort (hyperparams) to get the best information, masking algorithm of shap (1. POS tags of spacy)
# idea: masking multiword terms (phrase recognition then mask phrases)
# gensim phraser vs spacy chunks, 4/5-gram phrases to build thephraser , saffron
# get rid of stop words ('for')
# shap vs content words
# shap algorithms, how is shap permuting, masking
# get to the baseline (no supervision vs human supervision, alignment that will improve explainability)
# shap seed vs replicability, stability of the explainer
# collectio nwide processing, reducing it content words
# is it more pronounced on a training sample vs test sample (sensitivity to seen instances / features)
# syntehsizing or paraphrasing ("don't change the content" but change the wording as much as possible)
# sparsity of lexical
# content word selection for shap (papers)
shap_values[0]


# statistical improbable phrases (Multi word)

In [ ]:
shap.plots.text(shap_values)

In [ ]:
# Single class impact visualization
# https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/sentiment_analysis/Emotion%20classification%20multiclass%20example.html#Visualize-the-impact-on-a-single-class
shap.plots.text(shap_values[:,:, 6])

In [ ]:
# multiple instance bar plot per token

# Process the texts using SpaCy
docs = [spacy_model(sample) for sample in samples]

# Get the shap values for each processed document
shap_values = explainer([sample.text for sample in docs])

# Plot the shap values for each document
for i, doc in enumerate(docs):
    shap.plots.text(shap_values[i])

In [ ]:
shap.plots.bar(shap_values.abs.sum(0))